In [ ]:
from datetime import datetime
import pandas as pd
import sys
import os
from sqlalchemy import create_engine
from datetime import datetime
import logging

import check_server
from log_config import start_log
import organize_files as of

In [ ]:
logging.basicConfig(level=logging.DEBUG)

In [ ]:
SCHEMA_REF = {
    "Client Billing Descending":"client_billing","Fee Breakdown by Dept and Fee Earner":"fee_brkdn_dept_fe","Fee Summary by Dept and Fee Earner":"fee_smry_dept_fe",
    "Fees Billed":"fees_billed","Matter Source of Business inc Matter Bills (Bill Date)":"mttr_src_ref","Total Hours by Fee Earner-With Billings All":"tot_hrs_by_fe",
    "Matters Opened by FE":"mtrs_by_fe","Payment Received Analysis":"pmt_rcv_analysis"}

In [ ]:
def init_checks(info_file=None):
    all_details = {}
    info_dict = {}
    if info_file is not None:
        if os.path.exists(info_file):
            logging.info(f"Found file at [{info_file}]")
            all_details["loc"] = info_file
            #loc = info_file
            opened_file = open(all_details["loc"], encoding='utf8')
            from csv import reader
            read_file = reader(opened_file)
            db_creds = list(read_file)
            opened_file.close()

            for info in db_creds:
                info_dict[info[0]] = info[1]
            
            all_details['file_loc'] = info_dict["base_loc"]
            all_details['log_file'] = all_details['file_loc']+"\\Logs"
            all_details['final_files'] = all_details['file_loc']+"\\Final_Df"
            all_details['backup'] = all_details['file_loc']+"\\Backup"
            all_details['trigger_file'] = all_details['file_loc']+"\\file_trigger\\new_data_received.txt"
            init_logger = start_log(all_details['log_file'])

            imp_folds = ['file_loc','log_file','final_files','backup','trigger_file']
            for fold in imp_folds:
                if os.path.exists(all_details[fold]):
                    init_logger.info(f"Found {all_details[fold]}")
                else:
                    init_logger.error(f"Folder {all_details[fold]} Doesn't exist")

            all_details['db'] = info_dict["db_name"]
            all_details['db_user'] = info_dict["db_user"]
            all_details['db_password'] = info_dict["db_password"]
            all_details['db_host'] = info_dict["db_host"]
            all_details['db_port'] = info_dict["db_port"]

            return all_details
        else:
            logging.error("Important Variables not set.")
            logging.error("Exiting ...")
            sys.exit()
    else:
        i1 = check_server.chk_srvr()
        if i1.chk_base_dirs() != -1 & i1.chk_creds() != -1:
    
            logging.info("Checking Environment Variables for information")
            srv_dirs = i1.chk_base_dirs()
            db_creds = i1.chk_creds()

            if srv_dirs:
                all_details['file_loc'] = srv_dirs[0]
                all_details['log_file'] = srv_dirs[1]
                all_details['final_files'] = srv_dirs[2]
                all_details['backup'] = srv_dirs[3]
                all_details['trigger_file'] = srv_dirs[4]
                init_logger = start_log(all_details['log_file'])
                logging.info("Variables Set Successfully for File Locations")
            else:
                logging.error("Environment Variables for Files is not set")
                logging.info("Checking Environment Variables for DB Credentials ")

            if db_creds:
                all_details['db'] = db_creds[0]
                all_details['db_user'] = db_creds[1]
                all_details['db_password'] = db_creds[2]
                all_details['db_host'] = db_creds[3]
                all_details['db_port'] = db_creds[4]
                logging.info("Variables Set successfully for DB Creds ")
                return all_details
        else:
            logging.error("Environment Variables not set for DB Credentials.")
            return -1
        


In [ ]:
s_dict = init_checks("D:\Learning+Offline\db_creds.csv")
s_dict


In [ ]:
def main():
    import log_config

    if os.name == 'posix':
        print("Getting Details from the Environment Variables")
        all_info = init_checks()
    elif os.name == 'nt':
        print("Running on Windows")
        loc = input("Enter Location of Configuration file")
        if os.path.exists(loc):
            all_info = init_checks(loc)
        else:
            print(f"File [{loc}] not Found")

    main_logger = log_config.start_log(all_info["log_file"])    
    try:
        engine = create_engine(f'postgresql://{all_info["db_user"]}:{all_info["db_password"]}@{all_info["db_host"]}:{all_info["db_port"]}/{all_info["db"]}',echo=True)
    except:
        main_logger.error("Unable to connect to the Database")
    else:
        main_logger.info("Database Connection Established")

    dict_list = of.categorize_files(all_info["file_loc"])
    all_files = of.concat_files(dict_list, all_info["file_loc"], all_info["log_file"])



In [10]:
main()

nt
